# Using GN2's REST API

GN2 has a REST API through which the user can fetch data and perform several analyses.
This notebook will include examples for all the currently existing queries, as well as their parameters.

First, we'll start with necessary imports (urllib2 to do HTTP requests with Python and IPython.display to format JSON output).

In [1]:
from urllib.request import urlopen
import json
import uuid
from IPython.core.display import display, HTML

We'll also use a function that makes the displayed JSON collapsible, just for easier readability (written by Tarang Shah - https://gist.github.com/t27).

In [1]:
class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())
        # This line is missed out in most of the versions of this script across the web, it is essential for this to work interleaved with print statements
        self._ipython_display_()

    def _ipython_display_(self, level_to_show=2):
        display(HTML('<div id="{}" style="height: auto; width:100%;"></div>'.format(self.uuid)))
        display(HTML("""<script>
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_show_to_level(%s)
          document.getElementById('%s').appendChild(renderjson(%s))
        });</script>
        """ % (level_to_show, self.uuid, self.json_str)))

## Fetching Group/Dataset lists and information

The following query fetches the list of species available in the GN database:

In [1]:
response = urlopen("http://gn2-zach.genenetwork.org/api/v_pre1/species")
RenderJSON(json.loads(response.read()))

You can also get the information for a single species.

In [1]:
response = urlopen("http://gn2-zach.genenetwork.org/api/v_pre1/species/mouse")
RenderJSON(json.loads(response.read()))

Groups (for example RISets) can also be listed or selected individually in the same way.
To retrieve all groups across all species:

In [1]:
response = urlopen("http://gn2-zach.genenetwork.org/api/v_pre1/groups")
RenderJSON(json.loads(response.read()))

Or to retrieve all groups for mouse:

In [1]:
response = urlopen("http://gn2-zach.genenetwork.org/api/v_pre1/groups/mouse")
RenderJSON(json.loads(response.read()), level_to_show=1)

Individual group information can also be selected (with an optional field to limit the search to a species).
To get information on the HSNIH-Palmer rat group:

In [1]:
response = urlopen("http://gn2-zach.genenetwork.org/api/v_pre1/group/rat/HSNIH-Palmer")
RenderJSON(json.loads(response.read()))

The same can be done for datasets, but this query is limited to group.
To get a list of all datasets in the BXD group:

In [1]:
response = urlopen("http://gn2-zach.genenetwork.org/api/v_pre1/datasets/bxd")
RenderJSON(json.loads(response.read()))

To get information on a specific dataset, its Short_Abbreviation (from the previous query) can be used like the following:

In [1]:
response = urlopen("http://gn2-zach.genenetwork.org/api/v_pre1/dataset/bxd/HC_M2_0606_P")
RenderJSON(json.loads(response.read()))

Because of some quirks of the GN database, traits other than mRNA expression or genotype traits are treated different.
In this case, the user supplies the group name and trait ID (how to fetch these will be described later), and the dataset
query returns information about the publication it's from (like the title, description, pubmed_id, etc).

For example, to get publication information for BXD phenotype trait 10001:

In [1]:
response = urlopen("http://gn2.genenetwork.org/api/v_pre1/dataset/bxd/10001")
RenderJSON(json.loads(response.read()))

Trait information can also be selected by supplying the dataset abbreviation (for all traits) or the dataset abbreviation
and trait name (for individual traits). Trait information in this context includes things like the description, position, etc; a
separate query gets actual sample/strain data.

For example, to fetch information for all of the traits in the HC_M2_0606_P dataset (a BXD hippocampus dataset):

In [1]:
response = urlopen("http://gn2.genenetwork.org/api/v_pre1/traits/HC_M2_0606_P")
RenderJSON(json.loads(response.read()))

Or to fetch all the phenotype (non-mRNA expression/genotype) traits for a group, use the group name

In [1]:
response = urlopen("http://gn2.genenetwork.org/api/v_pre1/traits/HXBBXH")
RenderJSON(json.loads(response.read()))

Lastly, to view a single tria'ts information (name, gene symbol, description, location, etc):

In [1]:
response = urlopen("http://gn2.genenetwork.org/api/v_pre1/trait/HC_M2_0606_P/1436869_at")
RenderJSON(json.loads(response.read()))

## Downloading Sample Data and Genotypes